In [1]:
# Based on Tensorflow example notebook:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/multilayer_perceptron.ipynb

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

In [3]:
# # Generate tain and test data
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=8, 
                           n_redundant=0, n_clusters_per_class=2)
Y = np.array([Y, -(Y-1)]).T  # The model currently needs one column for each class
X, X_test, Y, Y_test = train_test_split(X, Y)

In [4]:
X.shape

(37500, 10)

In [5]:
Y.shape

(37500, 2)

In [6]:
data = pd.read_csv("dataset_4.csv")

X, X_test = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED)

Y = X['label']
Y = np.array([Y, -(Y-1)]).T
X = X.drop(['label'], axis=1)

Y_test = X_test['label']
Y_test = np.array([Y_test, -(Y_test-1)]).T
X_test = X_test.drop(['label'], axis=1)

X = X.values
X_test = X_test.values

In [7]:
X.shape

(93731, 27)

In [8]:
Y.shape

(93731, 2)

In [9]:
# Parameters
learning_rate = 0.001
training_epochs = 100
batch_size = 64
display_step = 1


# Network Parameters
n_hidden_1 = 20 # 1st layer number of features
n_hidden_2 = 20 # 2nd layer number of features
n_input = X.shape[1] # Number of features
n_classes = 2 # Number of classes to predict


# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [10]:
# Launch the graph

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X, total_batch)
        Y_batches = np.array_split(Y, total_batch)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))
    global result 
    result = tf.argmax(pred, 1).eval({x: X_test, y: Y_test})

('Epoch:', '0001', 'cost=', '39.783217147')
('Epoch:', '0002', 'cost=', '5.754569512')
('Epoch:', '0003', 'cost=', '3.799155140')
('Epoch:', '0004', 'cost=', '2.617563731')
('Epoch:', '0005', 'cost=', '2.082610326')
('Epoch:', '0006', 'cost=', '1.582036485')
('Epoch:', '0007', 'cost=', '1.198164364')
('Epoch:', '0008', 'cost=', '1.053796878')
('Epoch:', '0009', 'cost=', '0.876019726')
('Epoch:', '0010', 'cost=', '0.779691407')
('Epoch:', '0011', 'cost=', '0.805750388')
('Epoch:', '0012', 'cost=', '0.659128565')
('Epoch:', '0013', 'cost=', '0.624349342')
('Epoch:', '0014', 'cost=', '0.681898330')
('Epoch:', '0015', 'cost=', '0.540431968')
('Epoch:', '0016', 'cost=', '0.521798794')
('Epoch:', '0017', 'cost=', '0.607378880')
('Epoch:', '0018', 'cost=', '0.556043598')
('Epoch:', '0019', 'cost=', '0.513286958')
('Epoch:', '0020', 'cost=', '0.541102445')
('Epoch:', '0021', 'cost=', '0.464928531')
('Epoch:', '0022', 'cost=', '0.503092400')
('Epoch:', '0023', 'cost=', '0.452789272')
('Epoch:',